## Тестирование производительности OLAP-хранилищ Clickhouse и Vertica

В рамках исследования были рассмотрены Clickhouse и Vertica.

In [ ]:
!docker-compose up -d

#### Загрузка тестовых данных

Структура тестовой таблицы

| ----------- | ----------- | | id | UUID | | user_id | UUID | | movie_id | UUID | | viewed_frame | Int |

In [ ]:
!export PYTHONPATH="${PYTHONPATH}:${PWD}/../.."

#### Загрузка тестовых данных в Clickhouse

In [ ]:
from multiprocessing import Pool

import tqdm as tqdm
from clickhouse_driver import Client

from utils.fake_data_gen import Row, generate_fake_data
from config import CLICKHOUSE_HOST, NUMBER_OF_BATCHES, UPLOAD_BATCH_SIZE

client = Client(CLICKHOUSE_HOST)


def upload_batch(batch):
    columns = ', '.join(Row._fields)
    client.execute(
        f'INSERT INTO views ({columns}) VALUES',
        batch
    )

In [ ]:
test_data = generate_fake_data(UPLOAD_BATCH_SIZE, NUMBER_OF_BATCHES)

with Pool() as pool:
    r = list(tqdm.tqdm(
        pool.imap(upload_batch, test_data),
        total=NUMBER_OF_BATCHES
    ))

#### Загрузка тестовых данных в Vertica

In [ ]:
from multiprocessing import Pool

import tqdm
import vertica_python

from utils.fake_data_gen import Row, generate_fake_data
from config import NUMBER_OF_BATCHES, UPLOAD_BATCH_SIZE, VERTICA_CONNECTION_PARAMS


def upload_batch(batch):
    with vertica_python.connect(**VERTICA_CONNECTION_PARAMS) as connection:
        columns = ', '.join(Row._fields)
        placeholders = ', '.join(['%s'] * len(Row._fields))
        cursor = connection.cursor()
        cursor.executemany(
            f'INSERT INTO views ({columns}) VALUES ({placeholders})',
            batch
        )

In [ ]:
test_data = generate_fake_data(UPLOAD_BATCH_SIZE, NUMBER_OF_BATCHES)

with Pool() as pool:
    r = list(tqdm.tqdm(
        pool.imap(upload_batch, test_data),
        total=NUMBER_OF_BATCHES
    ))

#### Выполнение тестовых запросов

Использовались следующие тестовые запросы:

In [ ]:
from utils.test_queries import QUERIES

for name, query in QUERIES.items():
    print(f'{name}')
    print(f'{query}')
    print("----------")


#### Тестирование Clickhouse

In [ ]:
from clickhouse_driver import Client

from utils.test_queries import QUERIES
from utils.timer import timer
from config import BENCHMARK_ITERATIONS, CLICKHOUSE_HOST

client = Client(CLICKHOUSE_HOST)


@timer(BENCHMARK_ITERATIONS)
def execute_query(query: str):
    client.execute(query)

In [ ]:
for name, query in QUERIES.items():
    print(f'{name}')
    execute_query(query)

#### Тестирование Vertica

In [ ]:
import vertica_python

from utils.test_queries import QUERIES
from utils.timer import timer
from config import BENCHMARK_ITERATIONS, VERTICA_CONNECTION_PARAMS


@timer(BENCHMARK_ITERATIONS)
def execute_query(query: str):
    with vertica_python.connect(**VERTICA_CONNECTION_PARAMS) as connection:
        cursor = connection.cursor()
        cursor.execute(query)

In [ ]:
for name, query in QUERIES.items():
    print(f'{name}')
    execute_query(query)

#### Вывод

Clickhouse показывает лучшую производительность во всех проведенных тестах!

#### Остановка

In [ ]:
!docker-compose down -v